Notebook to process zebrafish data into the right format

In [1]:
import numpy as np
import pandas as pd

In [2]:
base_dir = 'C:/Users/joewa/Work/omics_integration/Transcriptomic, proteomic, and metabolomic landscape'

In [3]:
def load_data(base_dir, case, control):
    label = '%s_vs_%s' % (case, control)
    fname = base_dir + '/' + label + '.csv'
    df = pd.read_csv(fname, index_col=0)
    return df[['log2FoldChange', 'FDR']].rename(columns={
        'log2FoldChange': 'FC_%s' % label, 
        'FDR': 'padj_%s' % label
    })

In [4]:
distal_vs_middle = load_data(base_dir, 'distal', 'middle') 

In [5]:
distal_vs_proximal = load_data(base_dir, 'distal', 'proximal')

In [6]:
middle_vs_proximal = load_data(base_dir, 'middle', 'proximal')

In [7]:
df = pd.concat([distal_vs_middle, distal_vs_proximal, middle_vs_proximal], axis=1, sort=True)

In [12]:
df.to_csv(base_dir + '/' + 'data/gene_data.csv', index_label='Identifier')

Add columns to say which p-values are significant

In [45]:
T_TEST_THRESHOLD = 0.05
PADJ_COL_PREFIX = 'padj_'

In [51]:
data_df = df.copy()

In [49]:
if data_df is not None:
    padj_cols = list(filter(lambda x: x.startswith(PADJ_COL_PREFIX), data_df.columns))
    padj_col_count = len(padj_cols)
    if padj_col_count > 0:
        for index, row in data_df.iterrows():
            padj_values = row[padj_cols].values
            significant_all = False
            significant_any = False
            try:
                check = (padj_values > 0) & (padj_values < T_TEST_THRESHOLD)
                if len(check) > 0:
                    significant_all = np.all(check)
                    significant_any = np.any(check)
            except RuntimeWarning:
                print(row)
            data_df.loc[index, 'significant_all'] = bool(significant_all)
            data_df.loc[index, 'significant_any'] = bool(significant_any)

c:\users\joewa\.virtualenvs\web_omics-makc_z5x\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in greater
  if sys.path[0] == '':
c:\users\joewa\.virtualenvs\web_omics-makc_z5x\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in less
  if sys.path[0] == '':


In [42]:
type(data_df.head(10).to_dict()['significant_any']['ENSDARG00000000001'])

bool